# Portfolio aggregations
In this notebook we'll give some examples on how the portfolio aggregation methods can be used.

### Setting up
We'll set up the imports, data providers, and load the portfolio. For more details about this, please have a look at notebook one (getting started).

In [ ]:
# Only required if you didn't install the module yet
!pip install SBTi

In [ ]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetValidation
%aimport -pandas
import pandas as pd

In [ ]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
companies = portfolio[["company_id", "company_name"]].to_dict(orient="records")

company_data = SBTi.data.get_company_data([provider], companies)
target_data = SBTi.data.get_targets([provider], companies)
company_data = pd.merge(left=company_data,
                        right=portfolio[
                            [column
                                for column in portfolio.columns
                                if column not in ["company_name"]]],
                        how="left",
                        on=["company_id"])

target_validation = TargetValidation(target_data, company_data)
portfolio_data = target_validation.target_validation()

In [ ]:
temperature_score = TemperatureScore()
amended_portfolio = temperature_score.calculate(portfolio_data)

### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

In [ ]:
def print_aggregations(aggregations):
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

In [ ]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.WATS, []))

#### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 

In [ ]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.TETS, []))

#### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.

In [ ]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.MOTS, []))

#### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach. 

In [ ]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.EOTS, []))

#### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach. 

In [ ]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.ECOTS, []))

#### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach. 

In [ ]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.AOTS, []))